# SPECK32/64 Key Recovery: 1st bit

## Description

In this attack, we aim to recover the first bit of the key using a binary classifier. We train a neural network on plaintext-ciphertext pairs as samples and the first bit of the key as labels. We use SPECK32/64 with the smallest configuration (64-bit key and 32-bit blocks)

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import SPECKDatasetCiphertextPlaintextPairKey
from pipeline import *

2023-05-02 18:37:41.746833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [2]:
data = SPECKDatasetCiphertextPlaintextPairKey(64, 32, 'large')

train_labels, train_samples, test_labels, test_samples = data.get_data()

Error: dataset/assets/moby-dick.txt
Error: dataset/assets/shakespeare.txt


In [3]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (5782806,)
===== Label Shape: (64,)
===== Training Samples Shape: (5782806, 160)
===== Sample Shape: (160,)
===== Testing Labels Shape: (2478345,)
===== Testing Samples Shape: (2478345, 160)


### Preprocessing

We add 1 preprocessing step:
As our goal is only to recover the first bit of the key, we only keep the first bit of the labels to train our network.


In [4]:
train_samples = train_samples.astype(float)
test_samples = test_samples.astype(float)

In [5]:
train_labels_1st_bit = np.array([l[0] for l in train_labels]).astype(np.float32)
test_labels_1st_bit = np.array([l[0] for l in test_labels]).astype(np.float32)

In [6]:
get_dataset_info(train_labels_1st_bit, train_samples, test_labels_1st_bit, test_samples)

===== Training Labels Shape: (5782806,)
===== Label Shape: ()
===== Training Samples Shape: (5782806, 160)
===== Sample Shape: (160,)
===== Testing Labels Shape: (2478345,)
===== Testing Samples Shape: (2478345, 160)


## Creating the model

In [7]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

In [8]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

In [9]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = 1

# units per hidden layer
units = 128

# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.001,
#     decay_steps=10000,
#     decay_rate=-0.9)

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
metrics = ['binary_accuracy']
epochs = 50
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [10]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
# neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_bce, metrics=metrics)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               20608     
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 53,761
Trainable params: 53,761
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [11]:
history = train_model(neural_network, train_samples, train_labels_1st_bit, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/50
1157/1157 [==============================] - 19s 16ms/step - loss: 0.6933 - binary_accuracy: 0.5003
Epoch 2/50
1157/1157 [==============================] - 16s 13ms/step - loss: 0.6932 - binary_accuracy: 0.5011
Epoch 3/50
1157/1157 [==============================] - 16s 13ms/step - loss: 0.6931 - binary_accuracy: 0.5013
Epoch 4/50
1157/1157 [==============================] - 16s 13ms/step - loss: 0.6931 - binary_accuracy: 0.5015
Epoch 5/50
1157/1157 [==============================] - 16s 14ms/step - loss: 0.6931 - binary_accuracy: 0.5017
Epoch 6/50
1157/1157 [==============================] - 16s 14ms/step - loss: 0.6931 - binary_accuracy: 0.5015
Epoch 7/50
1157/1157 [==============================] - 16s 14ms/step - loss: 0.6931 - binary_accuracy: 0.5019
Epoch 8/50
1157/1157 [==============================] - 16s 14ms/step - loss: 0.6931 - binary_accuracy: 0.5019
Epoch 9/50
1157/1157 [==============================] - 16s 14ms/step - loss: 0.6931 - binary_accuracy: 0.5021
E

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [12]:
results = test_model_binary(neural_network, test_samples, test_labels_1st_bit, batch_size)

496/496 [==============================] - 3s 6ms/step


In [14]:
print("Correct predictions: " + str(results["correct_predictions"]))
print("Accuracy: " + str(results["accuracy"]))

Correct predictions: 1241671
Accuracy: 0.5010081324432232


In [15]:
total_1_bits = sum(test_labels_1st_bit)
distr = total_1_bits / len(test_labels_1st_bit)
print("'1' first bit distribution: " + str(distr))
print("'0' first bit distribution: "+ str(1-distr))

'1' first bit distribution: 0.5017057754267464
'0' first bit distribution: 0.49829422457325356


### Saving the model

In [16]:
save_model(neural_network, "speck32_64_1st_bit")